In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.entry_processed as entry_processed
import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.config as config

#add the parent folder to sys path so we can import from within the notebook subfolder
#sys.path.insert(0,'..')

print(sys.path)

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full -n 2 -fc'
#args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full"]
args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/testset/lns1full", "-k"]

PACKAGE_CONFIG='xfmkit/config.yaml'


In [ ]:
pxs, embedding, categories, classavg, palette, kde = entry_processed.read_processed(args_in)

kde

In [ ]:
val = embedding.shape[0]
val
np.log10(val)

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import scipy.stats as st
"""
data = np.random.multivariate_normal((0, 0), [[0.8, 0.05], [0.05, 0.7]], 100)
x = data[:, 0]
y = data[:, 1]
xmin, xmax = -3, 3
ymin, ymax = -3, 3

# Peform the kernel density estimate
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)
"""

Z_local = np.copy(kde.Z)
Z_local[Z_local < 0.00001] = -0.0001

fig = pl.figure()
ax = fig.gca()
#ax.set_xlim(xmin, xmax)
#ax.set_ylim(ymin, ymax)
# Contourf plot
cfset = ax.contourf(kde.X, kde.Y, Z_local, levels=25, cmap='Blues')
## Or kernel density estimate plot instead of the contourf plot
#ax.imshow(Z_local, cmap='Blues', extent=[-7, 20, -7, 20])
# Contour plot
#cset = ax.contour(kde.X, kde.Y, Z_local, colors='k')
# Label plot
#ax.clabel(cset, inline=1, fontsize=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')

pl.show()

In [ ]:
kde

In [ ]:
#get command line arguments
args = argops.readargs_processed(args_in)

image_directory=args.input_directory
output_directory=os.path.join(image_directory, "analysis")

if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

ds = processops.compile(image_directory)

ds.crop((args.x_coords[0], args.x_coords[1]), (args.y_coords[0], args.y_coords[1]))

pxs = structures.PixelSet(ds)

pxs.downsample_by_se()

pxs.apply_weights(amplify_list = args.amplify, 
                        suppress_list = args.suppress, 
                        ignore_list = args.ignore,
                        normalise = args.normalise, 
                        weight_transform = args.weight_transform, 
                        data_transform = args.data_transform 
                    )

In [ ]:
vis.tricolour_pixelset("Ca","Co","Si", pxs)



In [ ]:
pxs.weighted.mapview.shape

In [ ]:
import matplotlib.pyplot as plt

py = pxs.data.d[np.where(categories == 15)]
log = np.log(py, where=(py!=0))



index = utils.findelement(pxs.labels, "Fe")

print(index)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.hist(log[index], bins=20)

#ax.set_xscale('log')


In [ ]:
fig = vis.tricolour_pixelset("Fe", "S", "Cu", pxs)